In [1]:
import redis
redis.__version__

'5.0.8'

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Using smaller vector types

With the [Redis 7.4 release](https://redis.io/blog/announcing-redis-community-edition-and-redis-stack-74/) there is now support for bfloat16 and float16 data types in the vector store. Using this type is exactly the same as using float32 or other data types but will require a conversion if seeking to replace previously stored objects.

This tutorial will walk through repopulating and index as such.

## Packages

In [23]:
# NBVAL_SKIP
%pip install -q redis redisvl numpy sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


^C
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

##### if running local version make sure to double check it is updated >=7.4

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [37]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

In [2]:
# from redis import Redis
import redis

client = redis.Redis(
  host='redis-11118.c309.us-east-2-1.ec2.redns.redis-cloud.com',
  port=11118,
  password='qt1Zl62JL07F7RTGac40XCiEhksh4xjD')
# client = Redis.from_url(REDIS_URL)

In [3]:
client.ping()

True

# Define index and load data

We will be loading a schema of movie information with a 32bit float.

In [4]:
import json

with open("resources/movies.json", 'r') as file:
    movies = json.load(file)

In [5]:
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex

index_name = "movies32"

schema = IndexSchema.from_dict({
  "index": {
    "name": index_name,
  },
  "fields": [
    {
        "name": "title",
        "type": "text",
    },
    {
        "name": "description",
        "type": "text",
    },
    {
        "name": "genre",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "rating",
        "type": "numeric",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "vector",
        "type": "vector",
        "attrs": {
            "dims": 384,
            "distance_metric": "cosine",
            "algorithm": "hnsw",
            "datatype": "float32"
        }
    }
  ]
})


index = SearchIndex(schema, client)
index.create(overwrite=True, drop=True)

07:32:48 redisvl.index.index INFO   Index already exists, overwriting.


# Embed movie description vectors

In [6]:
import numpy as np
from sentence_transformers import SentenceTransformer

# load model for embedding our movie descriptions
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def embed_text(model, text):
    return np.array(model.encode(text)).astype(np.float32).tobytes()

/Users/robert.shelton/.pyenv/versions/3.11.9/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/robert.shelton/.pyenv/versions/3.11.9/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
# Note: convert embedding array to bytes for storage in Redis Hash data type
movie_data32 = [
    {
        **movie,
        "vector": embed_text(model, movie["description"])
    } for movie in movies
]

# Populate the index

In [8]:
keys = index.load(movie_data32)
keys

['rvl:37b42d13adaa4a4ea32db5e4d062126b',
 'rvl:9e3dfc7c67b744bb8199d364c60f8d89',
 'rvl:8f7758c7763842f78490e3c43aab1148',
 'rvl:c961854993a04d089da2e026033b80b3',
 'rvl:7f2699e7dbf3474283d34bf04b465f00',
 'rvl:cc56f14da1f8431c90a318167ce7e506',
 'rvl:5c0097099939492f87b2b56890bc7dfe',
 'rvl:6c215979299c424ebff07a4b85a60f43',
 'rvl:896cd78dae42483db66019f2b62a70e4',
 'rvl:b92104d038c04ab3beab80139f75e7d1',
 'rvl:17b08c1a64b743ecab2e5acaa4a777af',
 'rvl:d7e8d94d158e4003ad443ec5ce517f53',
 'rvl:a448ffc20ac8474ab776df5e54ae26a9',
 'rvl:87e2cd98e776411cb7f552a29a808a2a',
 'rvl:750ccddcf5604b12b33fb81e78042db0',
 'rvl:403fcaf607b04e5a9b7561a86082cb26',
 'rvl:12e4333184554daca7cbbf44e77ccb18',
 'rvl:5be2ed277c1c4f3b8afb929446afda9e',
 'rvl:6ccdd8c9eb1f433fa92c85f16d4351ff',
 'rvl:7ba0ee30d58f45e7bc1d5d21fbe9a45d']

In [9]:
client.memory_usage('rvl:37f6b0ea780c4a84a156f35f54a31c7e')

# Now let's convert to fp16

This is as simple as you think it is using numpy.

In [10]:
movie_data16 = []
for movie in movie_data32:
    vect16 = np.frombuffer(movie["vector"], dtype=np.float32).astype(np.float16)
    movie["vector"] = vect16.tobytes()
    movie_data16.append(movie)

In [11]:
movie_data16[0]

{'title': 'Explosive Pursuit',
 'genre': 'action',
 'rating': 7,
 'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.',
 'vector': b'\xe3+S\x18\xbd\x1d\xf6\xabs\x9eQ*\xfd)\xac$8\xb0{.\x80+o\xa8H\xaa3&\x0f+\xb2\xa0\x9f \xe5(O$n!\xfe\xa4\x18\xae\x92*\xbb\xa7\xba\xb0)\x88\xfc%\x17\x93E\xadg\xa7+)\x97\x9b:*),\x12\xa5\x83\xaeY\xaa\xa7\xa4G*1"a\'o\xac\xc3$\x1d-\x9b\xa9\xee\xad\xb9\x1f\xd8+f,0\xae\x82\xac\xe9\x1dg\xa8J\xa6\xe0\xa5\xa9\x9bT&T*U)\xaa\xaa\xb0\xa4\xc8(`"N(d-\xc7\xad\xe6\xac\xc4(3)\xc2\'H$\x94\x9f\xda\xaau\x9f\n\xaa\xcb\xa8\x93\x9f\xc6\x1bz"b\xa1u-h\xa1p\xad\xd7(n.\x81)\xf0\xa4Z*\x06)P\xaa\xf3\x9c\xbc\x17\xe8(\xf6&\xb3\x98\x83\x9b\xe7!\x1d\x85\x9a#2!\xc1 o\xa9\xc7$u*)-\xf0&\x9a p\x96\xe5\xa6*(\xad\'\x97\xa2m\xa1\xe7\nb/\xb8*\xb2\xa6\x99*\xf7\xaa\x8d$\x9f \xce\x9b|\xa9\x80#E\x9c\xef$\xb0-\x00\x808\x9d0\xab[\x9a\x81\x9c\xcc\x9d\xce\x9d[+h%C\xadc.\x07\xb1z\xa9\xbb\xa7\xb5+\x88#\xe7\xa8X$3\xa54$\xcc(%\xa7\xf3\xa2s\xa8\x1e\xa

# Create new index

In [12]:
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex

index_name = "movies16"

# this won't work till we merge Justin's stuff
schema = IndexSchema.from_dict({
  "index": {
    "name": index_name,
  },
  "fields": [
    {
        "name": "title",
        "type": "text",
    },
    {
        "name": "description",
        "type": "text",
    },
    {
        "name": "genre",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "rating",
        "type": "numeric",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "vector",
        "type": "vector",
        "attrs": {
            "dims": 384,
            "distance_metric": "cosine",
            "algorithm": "hnsw",
            "datatype": "float16"
        }
    }
  ]
})


index = SearchIndex(schema, client)
index.create(overwrite=True, drop=True)

ValidationError: 1 validation error for IndexSchema
__root__ -> attrs -> datatype
  value is not a valid enumeration member; permitted: 'FLOAT32', 'FLOAT64' (type=type_error.enum; enum_values=[<VectorDataType.FLOAT32: 'FLOAT32'>, <VectorDataType.FLOAT64: 'FLOAT64'>])

In [16]:
from redis.commands.search.field import VectorField, TagField, NumericField, TextField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

index_name = "movies"

schema = (
    VectorField(
        "vector",
        "HNSW",
        {
            "TYPE": "FLOAT16",
            "DIM": 384,
            "DISTANCE_METRIC": "COSINE"
        }
        ),
        NumericField("rating"),
        TagField("genre"),
        TextField("title"),
        TextField("description")
)

try:
    client.ft(index_name).info()
    print("Index exists!")
except:
    # index Definition
    definition = IndexDefinition(index_type=IndexType.HASH)

    # create Index
    client.ft(index_name).create_index(fields=schema, definition=definition)

In [17]:
def load_docs(client: redis.Redis, data: list[dict]):
    for i, d in enumerate(data):
        client.hset(
            i,
            mapping = d
        )

def print_results(res):
    docs = [(doc.title, doc.genre, doc.rating) for doc in res.docs]
    print(f"Top {len(docs)} movies: ", docs)

In [18]:
load_docs(client, movie_data16)

In [19]:
res = client.ft(index_name).search("*")
res

Result{20 total, docs: [Document {'id': '0', 'payload': None, 'title': 'Explosive Pursuit', 'genre': 'action', 'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.', 'vector': '+S\x18\x1dsQ*)$8{.+oH3&\x0f+ (O$n!\x18*)%\x17Eg+):*),\x12YG*1"a\'o$\x1d-\ueb79\x1f+f,0\x1dgJ३T&T*U)(`"N(d-ǭ(3)\'H$ڪu\n˨\x1bz"bu-hp(n.)Z*\x06)P\x17(&!\x1d#2! o$u*)-& p*(\'m\nb/**$ Λ|#E$-\x0080[̝Ν[+h%Cc.\x07z+#X$34$(%s\x1e\x1a`T/\x18q-\x1c,!\x0f%hH&.\x0c)1DJ&\x19.xf-\x1b\x0b\x1e\x18\x17/Q!(o(7(ԭ&\x1ez).,\x10){\'\x1f+B)\x1a\x15#%ܩ\x19%*(,짐$\x1c+L#p)7*\x00\x00%}8\x1e(\'*ߡkA)6.(/v!J((Zhkˠ\x1c,$ˢ\x0c+\x1dʨ+,Kˡw\x04-$đD!^*&:@"%\x0c(ު\x1f {<A&,7+/y");%\x16T,Nno)*-i+s,%-Uj$sY+\x1d\'$b\x1c\x1d-#?\x00W+?)\'+,B?W#+\x0f\'("&R #h,4i|.ks+*\x19!m\x1b\x04\\;*U.ש"\x1es-\x19J,\x11+N@\'&\x1b)\\\x0b wP+ת)\t$m*/ ⫽#?\x039(L\',', 'rating': '7'}, Document {'id': '1', 'payload': None, 'title': 'Skyfall', 'genre': 'action', 'description': 'James Bond returns to track down a dange

In [21]:
res.docs

[Document {'id': '0', 'payload': None, 'title': 'Explosive Pursuit', 'genre': 'action', 'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.', 'vector': '+S\x18\x1dsQ*)$8{.+oH3&\x0f+ (O$n!\x18*)%\x17Eg+):*),\x12YG*1"a\'o$\x1d-\ueb79\x1f+f,0\x1dgJ३T&T*U)(`"N(d-ǭ(3)\'H$ڪu\n˨\x1bz"bu-hp(n.)Z*\x06)P\x17(&!\x1d#2! o$u*)-& p*(\'m\nb/**$ Λ|#E$-\x0080[̝Ν[+h%Cc.\x07z+#X$34$(%s\x1e\x1a`T/\x18q-\x1c,!\x0f%hH&.\x0c)1DJ&\x19.xf-\x1b\x0b\x1e\x18\x17/Q!(o(7(ԭ&\x1ez).,\x10){\'\x1f+B)\x1a\x15#%ܩ\x19%*(,짐$\x1c+L#p)7*\x00\x00%}8\x1e(\'*ߡkA)6.(/v!J((Zhkˠ\x1c,$ˢ\x0c+\x1dʨ+,Kˡw\x04-$đD!^*&:@"%\x0c(ު\x1f {<A&,7+/y");%\x16T,Nno)*-i+s,%-Uj$sY+\x1d\'$b\x1c\x1d-#?\x00W+?)\'+,B?W#+\x0f\'("&R #h,4i|.ks+*\x19!m\x1b\x04\\;*U.ש"\x1es-\x19J,\x11+N@\'&\x1b)\\\x0b wP+ת)\t$m*/ ⫽#?\x039(L\',', 'rating': '7'},
 Document {'id': '1', 'payload': None, 'title': 'Skyfall', 'genre': 'action', 'description': 'James Bond returns to track down a dangerous new enemy who thr

In [ ]:
# Does work with pure redis client.
client.memory_usage('rvl:37f6b0ea780c4a84a156f35f54a31c7e')